# Segmenter
The Segmenter abstract class defines the interface for implementations that segment raw response text into response units. Concrete implementations of this class can use various approaches such as rule-based, machine learning, or natural language processing techniques.

In [ ]:
#| default_exp segmenting

In [ ]:
#| export
from abc import ABC, abstractmethod
from typing import List, Tuple, Dict

class Segmenter(ABC):
    """
    Abstract base class for implementing text segmentation strategies.
    """

    @abstractmethod
    def __call__(self, text: str) -> List[Dict[str, object]]:
        """
        Splits the input text into a list of response units.

        Parameters:
            text (str): The input text to be segmented.

        Returns:
            List[Dict[str, object]]: The list of segmented response units.
            
            Each unit is a dictionary with keys "text" and "spans":
            The "text" key corresponds to the text representation, 
            The "spans" key corresponds to a list of character-level start and end indices (tuples) in the input text.
        """
        pass

## Sentence Segmenter
A baseline concrete class that applies `spacy`'s built-in sentence tokenizer to segment text into response units.

In [ ]:
#| export
import spacy

class SentenceSegmenter(Segmenter):

    """
    Concrete Segmenter class that identifies the sentences in the input text using the SpaCy library.
    """

    def __init__(self):
        self._nlp = spacy.load("en_core_web_sm")

    def __call__(self, text: str) -> List[Dict[str, object]]:
        """
        Splits the input text into a list of response units.

        Parameters:
            text (str): The input text to be segmented.

        Returns:
            List[Dict[str, object]]: The list of segmented response units.
            
            Each unit is a dictionary with keys "text" and "spans":
            The "text" key corresponds to the text representation, 
            The "spans" key corresponds to a list of character-level start and end indices (tuples) in the input text.
        """
        return [{"text": sent.text, "spans": [(sent.start_char, sent.end_char)]} for sent in self._nlp(text).sents]

In [ ]:
segmenter = SentenceSegmenter()

text = "One fine day an old Maine man was fishing on his favorite lake and catching very little. Finally, he gave up and walked back along the shore to his fishing shack. When he got close to the front door, he saw it was open. "
result = segmenter(text)

result

[{'text': 'One fine day an old Maine man was fishing on his favorite lake and catching very little.',
  'spans': [(0, 88)]},
 {'text': 'Finally, he gave up and walked back along the shore to his fishing shack.',
  'spans': [(89, 162)]},
 {'text': 'When he got close to the front door, he saw it was open.',
  'spans': [(163, 219)]}]

## AllSentenceFragmentsSegmenter

In [ ]:
#| export

import spacy
from typing import List
from spacy.tokens import Span

class AllSentenceFragmentsSegmenter(Segmenter):

    """
    Concrete Segmenter class that generates all possible segments of tokens in each sentence.
    """

    def __init__(self):
        self.nlp = spacy.load("en_core_web_sm")

    def __call__(self, text: str) -> List[List[Tuple[str, Tuple[int, int]]]]:
        """
        Splits the input text into a list of response units.

        Parameters:
            text (str): The input text to be segmented.

        Returns:
            List[Dict[str, object]]: The list of segmented response units.
            
            Each unit is a dictionary with keys "text" and "spans":
            The "text" key corresponds to the text representation, 
            The "spans" key corresponds to a list of character-level start and end indices (tuples) in the input text.
        """
        segments = []
        for sentence in self.nlp(text).sents:
            for i in range(len(sentence)):
                for j in range(i + 1, len(sentence) + 1):

                    sentence_fragment = sentence[i:j]
                    if sentence_fragment.text.strip():
                        segments.append({
                            "text": sentence_fragment.text,
                            "spans": [(sentence_fragment.start_char, sentence_fragment.end_char)]
                        })

        return segments

In [ ]:
text = "One fine day an old Maine man was fishing "
segmenter = AllSentenceFragmentsSegmenter()
segmenter(text)

[{'text': 'One', 'spans': [(0, 3)]},
 {'text': 'One fine', 'spans': [(0, 8)]},
 {'text': 'One fine day', 'spans': [(0, 12)]},
 {'text': 'One fine day an', 'spans': [(0, 15)]},
 {'text': 'One fine day an old', 'spans': [(0, 19)]},
 {'text': 'One fine day an old Maine', 'spans': [(0, 25)]},
 {'text': 'One fine day an old Maine man', 'spans': [(0, 29)]},
 {'text': 'One fine day an old Maine man was', 'spans': [(0, 33)]},
 {'text': 'One fine day an old Maine man was fishing', 'spans': [(0, 41)]},
 {'text': 'fine', 'spans': [(4, 8)]},
 {'text': 'fine day', 'spans': [(4, 12)]},
 {'text': 'fine day an', 'spans': [(4, 15)]},
 {'text': 'fine day an old', 'spans': [(4, 19)]},
 {'text': 'fine day an old Maine', 'spans': [(4, 25)]},
 {'text': 'fine day an old Maine man', 'spans': [(4, 29)]},
 {'text': 'fine day an old Maine man was', 'spans': [(4, 33)]},
 {'text': 'fine day an old Maine man was fishing', 'spans': [(4, 41)]},
 {'text': 'day', 'spans': [(9, 12)]},
 {'text': 'day an', 'spans': [(9, 1

## MultiSentenceFragmentSegmenter

In [ ]:
#| export
import spacy
from typing import List, Tuple
from spacy.tokens import Doc

class MultiSentenceFragmentsSegmenter(Segmenter):

    """
    Concrete Segmenter class that generates all possible segments of tokens
    within a specified number of sentences in the input text.
    """

    def __init__(self, max_sentences: int, min_tokens: int = 1):
        self.nlp = spacy.load("en_core_web_sm")
        self.nlp.Defaults.stop_words.add("um")
        self.nlp.Defaults.stop_words.add("uh")

        self.max_sentences = max_sentences
        self.min_tokens = min_tokens

    def __call__(self, text: str) -> List[Dict[str, object]]:
        """
        Splits the input text into a list of response units.

        Parameters:
            text (str): The input text to be segmented.

        Returns:
            List[Dict[str, object]]: The list of segmented response units.
            
            Each unit is a dictionary with keys "text" and "spans":
            The "text" key corresponds to the text representation, 
            The "spans" key corresponds to a list of character-level start and end indices (tuples) in the input text.
        """
        segments = []
        doc = self.nlp(text)
        sentences = list(doc.sents)
        num_sentences = len(sentences)

        for start_idx in range(num_sentences):
            for end_idx in range(start_idx + 1, min(start_idx + self.max_sentences + 1, num_sentences + 1)):
                segment = doc[sentences[start_idx].start:sentences[end_idx - 1].end]

                for i in range(len(segment)):
                    for j in range(i + 1, len(segment) + 1):

                        fragment = segment[i:j]

                        token_count = sum(not (token.is_punct or token.is_stop) for token in fragment)

                        if token_count >= self.min_tokens and fragment.text.strip():
                            segments.append({
                                "text": fragment.text,
                                "spans": [(fragment.start_char, fragment.end_char)]
                            })

        return segments


In [ ]:
text = "This is the first sentence. THAT IS THE SECOND SENTENCE. HeRe Is ThE ThIrD SeNtEnCe."
segmenter = MultiSentenceFragmentsSegmenter(max_sentences=1, min_tokens=1)
segmenter(text)

[{'text': 'This', 'spans': [(0, 4)]},
 {'text': 'This is', 'spans': [(0, 7)]},
 {'text': 'This is the', 'spans': [(0, 11)]},
 {'text': 'This is the first', 'spans': [(0, 17)]},
 {'text': 'This is the first sentence', 'spans': [(0, 26)]},
 {'text': 'This is the first sentence.', 'spans': [(0, 27)]},
 {'text': 'is', 'spans': [(5, 7)]},
 {'text': 'is the', 'spans': [(5, 11)]},
 {'text': 'is the first', 'spans': [(5, 17)]},
 {'text': 'is the first sentence', 'spans': [(5, 26)]},
 {'text': 'is the first sentence.', 'spans': [(5, 27)]},
 {'text': 'the', 'spans': [(8, 11)]},
 {'text': 'the first', 'spans': [(8, 17)]},
 {'text': 'the first sentence', 'spans': [(8, 26)]},
 {'text': 'the first sentence.', 'spans': [(8, 27)]},
 {'text': 'first', 'spans': [(12, 17)]},
 {'text': 'first sentence', 'spans': [(12, 26)]},
 {'text': 'first sentence.', 'spans': [(12, 27)]},
 {'text': 'sentence', 'spans': [(18, 26)]},
 {'text': 'sentence.', 'spans': [(18, 27)]},
 {'text': 'THAT', 'spans': [(28, 32)]},
 {'

## AllFragmentsSegmenter

In [ ]:
#| export

import spacy
from typing import List, Tuple

class AllFragmentsSegmenter:
    """
    Concrete Segmenter class that generates all possible segments of tokens in the entire text.
    """

    def __init__(self):
        self.nlp = spacy.load("en_core_web_sm")

    def __call__(self, text: str) -> List[Tuple[str, Tuple[int, int]]]:
        """
        Splits the input text into a list of response units.

        Parameters:
            text (str): The input text to be segmented.

        Returns:
            List[Tuple[str, Tuple[int, int]]]: The list of segmented response units.
            
            Each unit is a tuple with the following elements:
            - The text representation of the fragment.
            - A tuple representing the character-level start and end indices of the fragment in the input text.
        """
        segments = []
        doc = self.nlp(text)

        # Iterate over all possible fragments of tokens in the entire text
        for i in range(len(doc)):
            for j in range(i + 1, len(doc) + 1):
                fragment = doc[i:j]
                if fragment.text.strip():
                    segments.append({'text': fragment.text, 'spans':[(fragment.start_char, fragment.end_char)]})

        return segments

In [ ]:
text = "One fine day an old Maine man was fishing "
segmenter = AllFragmentsSegmenter()
segmenter(text)

[('One', [(0, 3)]),
 ('One fine', [(0, 8)]),
 ('One fine day', [(0, 12)]),
 ('One fine day an', [(0, 15)]),
 ('One fine day an old', [(0, 19)]),
 ('One fine day an old Maine', [(0, 25)]),
 ('One fine day an old Maine man', [(0, 29)]),
 ('One fine day an old Maine man was', [(0, 33)]),
 ('One fine day an old Maine man was fishing', [(0, 41)]),
 ('fine', [(4, 8)]),
 ('fine day', [(4, 12)]),
 ('fine day an', [(4, 15)]),
 ('fine day an old', [(4, 19)]),
 ('fine day an old Maine', [(4, 25)]),
 ('fine day an old Maine man', [(4, 29)]),
 ('fine day an old Maine man was', [(4, 33)]),
 ('fine day an old Maine man was fishing', [(4, 41)]),
 ('day', [(9, 12)]),
 ('day an', [(9, 15)]),
 ('day an old', [(9, 19)]),
 ('day an old Maine', [(9, 25)]),
 ('day an old Maine man', [(9, 29)]),
 ('day an old Maine man was', [(9, 33)]),
 ('day an old Maine man was fishing', [(9, 41)]),
 ('an', [(13, 15)]),
 ('an old', [(13, 19)]),
 ('an old Maine', [(13, 25)]),
 ('an old Maine man', [(13, 29)]),
 ('an old Mai

## ClausiePropositionSegmenter
The `ClausiePropositionSegmenter` class uses the `spacy-clausie` library to segment sentences into propositions. `clausie` uses a rule-based approach to identify propositions within sentences. It uses a set of hand-crafted rules to identify the subject, verb, and object of a proposition.

In [ ]:
#| export
import spacy
import claucy
from typing import List


class ClausiePropositionSegmenter(Segmenter):

    """
    Concrete Segmenter class that generates propositions from sentences using spacy-clausie.
    """

    def __init__(self):
        self._nlp = spacy.load("en_core_web_sm")
        claucy.add_to_pipe(self._nlp)

    def __call__(self, text: str) -> List[List[Tuple[str, Tuple[int, int]]]]:
        """
        Splits the input text into a list of response units.

        Parameters:
            text (str): The input text to be segmented.

        Returns:
            List[Dict[str, object]]: The list of segmented response units.
            
            Each unit is a dictionary with keys "text" and "spans":
            The "text" key corresponds to the text representation, 
            The "spans" key corresponds to a list of character-level start and end indices (tuples) in the input text.
        """
        propositions = []

        for sent in self._nlp(text).sents:
            for clause in sent._.clauses:
                for prop in clause.to_propositions(inflect=None):
                    propositions.append({
                        "text": " ".join([p.text for p in prop]),
                        "spans": [(p.start_char, p.end_char) for p in prop]
                    })

        return propositions

In [ ]:
segmenter = ClausiePropositionSegmenter()

text = "One fine day an old Maine man was fishing on his favorite lake and catching very little. Finally, he gave up and walked back along the shore to his fishing shack. When he got close to the front door, he saw it was open. "

result = segmenter(text)

result

[{'text': 'an old Maine man was fishing on his favorite lake',
  'spans': [(13, 29), (30, 41), (42, 62)]},
 {'text': 'an old Maine man catching very little',
  'spans': [(13, 29), (67, 75), (76, 87)]},
 {'text': 'he gave Finally', 'spans': [(98, 100), (101, 105), (89, 96)]},
 {'text': 'he walked back along the shore to his fishing shack',
  'spans': [(98, 100), (113, 119), (120, 140), (141, 161)]},
 {'text': 'he walked back along the shore',
  'spans': [(98, 100), (113, 119), (120, 140)]},
 {'text': 'he walked to his fishing shack',
  'spans': [(98, 100), (113, 119), (141, 161)]},
 {'text': 'he got When close to the front door',
  'spans': [(168, 170), (171, 174), (163, 167), (175, 198)]},
 {'text': 'he got close to the front door',
  'spans': [(168, 170), (171, 174), (175, 198)]},
 {'text': 'he got When', 'spans': [(168, 170), (171, 174), (163, 167)]},
 {'text': 'he saw was', 'spans': [(200, 202), (203, 206), (210, 213)]},
 {'text': 'it was open', 'spans': [(207, 209), (210, 213), (21

## CTRL44 Sentence Simplifier

In [ ]:
#| export

from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from spacy.tokens.span import Span
import spacy
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import logging
logging.getLogger('sentence_transformers').setLevel(logging.ERROR)

def map_segments_to_original_text(
        original_text: str, segments: List[str], start: int,
        model_name: str = "sentence-transformers/paraphrase-distilroberta-base-v1") -> List[Tuple[int, int]]:
    """
    Uses maximum similarity matching to map the segment to a span the original sentence.

    Parameters:
        original_text (str): The original sentence.
        segments (List[Span]): The list of segments of the original sentence.
        model_name (str): The name of the sentence embedding model.

    Returns:
        List[Tuple[int, int]]: The list of character-level start and end indices (tuples) in the original sentence.
    """
    
    model = SentenceTransformer(model_name)
    nlp = spacy.load("en_core_web_sm")
    original_doc = nlp(original_text)

    all_segments = []
    for i in range(len(original_text)):
        for j in range(i + 1, len(original_doc) + 1):
            all_segments.append(original_doc[i:j])

    segment_embeddings = np.array(model.encode([s.text for s in all_segments]))
    simplified_sentence_embeddings = np.array(model.encode(segments))
    similarity_matrix = cosine_similarity(simplified_sentence_embeddings, segment_embeddings)

    # Find the most similar segment for each simplified sentence
    most_similar_segment_indices = np.argmax(similarity_matrix, axis=1)
    simplified_to_original_mapping = [all_segments[index] for index in most_similar_segment_indices]

    return [(s.start_char + start, s.end_char + start) for s in simplified_to_original_mapping]

class SimpleSentenceSegmenter(Segmenter):
    """
    Implementation of the Segmenter abstract base class that recursively segments text into sentences using spacy,
    and generates the simplest possible sentences using the ctrl44-simp model.
    """

    def __init__(self, model_name: str = "liamcripwell/ctrl44-simp", depth_limit=1, sep_type="<ssplit>"):
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        self.model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
        self.nlp = spacy.load("en_core_web_sm")
        self.depth_limit = depth_limit
        self.sep_type = sep_type

    def __call__(self, text: str) -> List[Dict[str, object]]:
        """
        Splits the input text into a list of response units.

        Parameters:
            text (str): The input text to be segmented.

        Returns:
            List[Dict[str, object]]: The list of segmented response units.
            
            Each unit is a dictionary with keys "text" and "spans":
            The "text" key corresponds to the text representation, 
            The "spans" key corresponds to a list of character-level start and end indices (tuples) in the input text.
        """
        segments = []
        for sentence in self.nlp(text).sents:
            simplified_sentences = self._simplify_sentence_recursive(sentence, depth=1)
            spans = map_segments_to_original_text(sentence.text, [s.text for s in simplified_sentences], start=sentence.start_char)

            segments.extend([{"text": s.text, "spans": [spans[i]]} for i, s in enumerate(simplified_sentences)])

        return segments

    def _simplify_sentence_recursive(self, sentence: Span, depth) -> List[Span]:
        """
        Recursively simplifies the input sentence using the specfied model.

        Parameters:
            sentence (Span): The input sentence to be simplified.
            depth (int): The current depth of the recursion.

        Returns:
            List[Span]: The list of simplified sentences.
        """

        # probes model to produce a string containing at least one sentence
        inputs = self.tokenizer(self.sep_type + sentence.text, return_tensors="pt")
        outputs = self.model.generate(**inputs, num_beams=10, max_length=128)
        output_string = self.tokenizer.decode(outputs[0], skip_special_tokens=True)

        # splits the output string into sentences
        generated_sentences = [sent for sent in self.nlp(output_string).sents if sent.text.upper().isupper()]

        # if the output string contains only one sentence, or the sentence is unchanged, return the sentence
        if len(generated_sentences) == 1 or sentence.text in [sent.text for sent in generated_sentences]:
            return [sentence]
        elif depth == self.depth_limit:
            return generated_sentences
        else:
            simplest_sentences = []
            for generated_sentence in generated_sentences:
                simplest_sentences.extend(self._simplify_sentence_recursive(generated_sentence, depth=depth+1))
            return simplest_sentences

In [ ]:
text = """One fine day an old Maine man was fishing on his favorite lake and catching very little. Finally, he gave up and walked back along the shore to his fishing shack. When he got close to the front door, he saw it was open. Being of a suspicious nature, he walked to the door quietly and looked inside. There was a big black bear. It was just pulling the cork out of his molasses jug with its teeth. The molasses spilled all over the floor and the bear rubbed his paw in it, smearing it all over.
"""

segmenter = SimpleSentenceSegmenter(depth_limit=1)
result = segmenter.__call__(text)
result

[{'text': ' One fine day an old Maine man was fishing on his favorite lake.',
  'spans': [(0, 62)]},
 {'text': 'He caught very little.', 'spans': [(63, 88)]},
 {'text': ' Finally, he gave up.', 'spans': [(89, 108)]},
 {'text': 'He walked back along the shore to his fishing shack.',
  'spans': [(109, 162)]},
 {'text': 'When he got close to the front door.', 'spans': [(163, 199)]},
 {'text': 'He saw it was open.', 'spans': [(200, 219)]},
 {'text': ' Being of a suspicious nature.', 'spans': [(220, 248)]},
 {'text': 'He walked to the door quietly and looked inside.',
  'spans': [(250, 298)]},
 {'text': 'There was a big black bear.', 'spans': [(299, 326)]},
 {'text': ' It was just pulling the cork out of his molasses jug.',
  'spans': [(327, 379)]},
 {'text': 'with its teeth.', 'spans': [(380, 395)]},
 {'text': ' The molasses spilled all over the floor.', 'spans': [(396, 439)]},
 {'text': 'The bear rubbed his paw in it, smearing it all over.',
  'spans': [(440, 492)]}]

## OIE6 Proposition Segmenter


In [ ]:
import subprocess
import os

def run_command_in_conda_env(conda_env, command, working_dir=None):
    conda_path = os.environ.get("CONDA_PREFIX")
    if conda_path is None:
        raise ValueError("Conda path not found. Please ensure you have installed Conda properly.")
        
    if os.name == "nt":  # Windows
        activate_command = f"{conda_path}\\Scripts\\activate.bat {conda_env} && {command}"
    else:  # POSIX
        activate_command = f"conda activate {conda_env} && {command}"
    
    process = subprocess.run(activate_command, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True, cwd=working_dir)
    
    if process.returncode != 0:
        print(f"An error occurred while executing the command in the conda environment '{conda_env}':")
        print(process.stderr)
        return None
    
    return process.stdout

class OIE6PropositionSegmenter(Segmenter):
    def __init__(self, working_dir="D:/openie6", conda_env="openie6"):
        self._nlp = spacy.load("en_core_web_sm")
        self.conda_env = "openie6"
        self.command = (
            "python run.py --mode predict --inp sentences.txt --out predictions.txt --rescoring --task oie --gpus 0 "
            "--oie_model models/oie_model/epoch=14_eval_acc=0.551_v0.ckpt "
            "--conj_model models/conj_model/epoch=28_eval_acc=0.854.ckpt "
            "--rescore_model models/rescore_model"
        )
        self.working_dir = working_dir

    def __call__(self, text: str) -> List[str]:
        """
        Splits the input text into a list of sentences using the SpaCy library and
        further segments each sentence by proposition using the OpenIE4 library.

        Parameters:
            text (str): The input text to be segmented.

        Returns:
            List[str]: The list of segmented propositions as response units.
        """
        doc = self._nlp(text)
        sentences = [sent.text for sent in doc.sents]
        with open(os.path.join(self.working_dir, "sentences.txt"), "w") as f:
            f.write("\n".join(sentences))

        run_command_in_conda_env(self.conda_env, self.command, self.working_dir)

        with open(os.path.join(self.working_dir, "predictions.txt.oie"), "r") as f:
            output = f.read().strip().split('\n\n')

        propositions = []
        for sentence_output in output:
            sentence_propositions = sentence_output.split('\n')
            if len(sentence_propositions) == 1:
                if len(sentence_propositions[0].strip()) > 0:
                    propositions.append(sentence_propositions[0].strip())
                continue
            for line in sentence_propositions[1:]:
                proposition = line[len('1.00: ('):-1].strip() #.replace('; ', '...'))
                if len(proposition) > 0:
                    propositions.append(proposition)
                
        return propositions

In [ ]:
#|slow
segmenter = OIE6PropositionSegmenter()
segmenter(text)

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()